In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
from pavlov import stats
stats.plot(['2021-01-15 23-51-09 early-mover 7x7-entropy', '*known-kicks', -1], 'elo-mohex', 'μ', skip=1, fill=True)

In [ ]:
from pavlov import stats
stats.from_run_sync(-1, rule='60s')

In [ ]:
%debug

In [ ]:
stats.review('*known-kicks', channel='mcts-n-leaves')

In [ ]:
%debug